In [10]:
# work2
# 1.数据集：使用AI Studio平台作业项目中的/datasets/image中的数据集。

# 2.图像数据处理方式：

# 使用高斯、瑞利、伽马、均匀和椒盐等噪声中的一种方法处理原图片，查看噪声处理图片后的效果；
# 使用邻域平均法、中值滤波、最大值滤波、最小值滤波和高斯滤波等方法中的一种处理加过噪声后的图像，完成去噪声操作；
# 使用RGB、GRAY（灰度图像）、XYZ、YCrC、HSV、HLS、CIELab、CIELuv或者Bayer等色彩空间处理第二步处理过的图像，如将RGB图像转化成GRAY灰度图像；
# 将上面第三步得到的图片作为模型输入，使用普通CNN网络、ResNet或者VGG模型提取处理过程中的特征图，并将特征图保存。

# 结果保存在/result/image中

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import paddle
from paddle.vision.models import resnet50, vgg16
from scipy.ndimage import gaussian_filter, median_filter, maximum_filter, minimum_filter

In [12]:
# 加载数据集中的图像
def load_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 转换为RGB格式
    return image

In [13]:
# 对图像添加高斯噪声
def add_gaussian_noise(image, mean=0, sigma=50):
    noise = np.random.normal(mean, sigma, image.shape).astype(np.float32)
    noisy_image = image + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

In [14]:
# 去噪声 - 高斯滤波
def denoise_image(image, method="gaussian", kernel_size=5):
    if method == "gaussian":
        return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    elif method == "median":
        return median_filter(image, size=kernel_size)
    elif method == "maximum":
        return maximum_filter(image, size=kernel_size)
    elif method == "minimum":
        return minimum_filter(image, size=kernel_size)
    else:
        raise ValueError("Unsupported denoise method!")

In [15]:
# 转换图像色彩空间 - 例如将RGB转换为灰度图像
def convert_color_space(image, space="GRAY"):
    if space == "GRAY":
        return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    elif space == "HSV":
        return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    elif space == "XYZ":
        return cv2.cvtColor(image, cv2.COLOR_RGB2XYZ)
    elif space == "LAB":
        return cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    else:
        raise ValueError("Unsupported color space!")

In [16]:
# 使用CNN模型（ResNet或者VGG）提取特征
def extract_features(image, model_name="resnet50"):
    """
    提取给定图像的特征，使用ResNet50或VGG16等模型。
    """
    # 如果图像是灰度图，将其转为三通道
    if len(image.shape) == 2:  
        image = np.stack([image] * 3, axis=-1)  # 灰度图转RGB
    
    # 将图像转为float32类型并进行归一化
    image = image.astype('float32') / 255.0
    image = np.transpose(image, (2, 0, 1))  # 调整通道顺序为 [C, H, W]
    image = np.expand_dims(image, axis=0)  # 增加batch维度
    
    # 使用PaddlePaddle的ResNet或VGG模型
    if model_name == "resnet50":
        model = resnet50(pretrained=True)
    elif model_name == "vgg16":
        model = vgg16(pretrained=True)
    else:
        raise ValueError("Unsupported model!")
    
    model.eval()  # 设置为评估模式
    # 将图像转换为Paddle tensor
    image_tensor = paddle.to_tensor(image)
    
    # 提取特征
    features = model(image_tensor)
    return features.numpy()

In [17]:
# 保存处理后的图像
def save_image(image, file_path, cmap='gray'):
    plt.imshow(image, cmap=cmap)
    plt.axis('off')
    plt.savefig(file_path)
    plt.close()

In [18]:
# 主流程 - 处理目录中的所有图像
def process_images_in_directory(input_directory, output_directory):
    # 获取目录下所有图像文件
    image_files = [f for f in os.listdir(input_directory) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    for image_file in image_files:
        # 构造完整输入文件路径
        file_path = os.path.join(input_directory, image_file)
        
        # 加载图像
        image = load_image(file_path)
        
        # 添加高斯噪声
        noisy_image = add_gaussian_noise(image)
        
        # 去噪声操作
        denoised_image = denoise_image(noisy_image, method="gaussian")
        
        # 转换色彩空间
        gray_image = convert_color_space(denoised_image, space="GRAY")
        
        # 为每张图像创建一个单独的文件夹
        image_name = os.path.splitext(image_file)[0]  # 获取图像的名称（去除扩展名）
        image_output_dir = os.path.join(output_directory, image_name)
        os.makedirs(image_output_dir, exist_ok=True)  # 创建图像对应的文件夹
        
        # 保存噪声图像、去噪图像、灰度图像
        save_image(noisy_image, os.path.join(image_output_dir, f"noisy_{image_name}.png"))
        save_image(denoised_image, os.path.join(image_output_dir, f"denoised_{image_name}.png"))
        save_image(gray_image, os.path.join(image_output_dir, f"gray_{image_name}.png"))

In [19]:
# 运行主程序
if __name__ == '__main__':
    input_directory = '/home/aistudio/datasets/image/'  # 输入图像所在的目录
    output_directory = '/home/aistudio/result/image/'  # 输出图像存储的根目录
    process_images_in_directory(input_directory, output_directory)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
